In [1]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("Final/dataset_columns.csv")
df = df[['Image Index', 'Finding Labels']].dropna()
df['Image Index'] = df['Image Index'].str.strip().str.lower()
df['Finding Labels'] = df['Finding Labels'].str.strip()

image_labels = dict(zip(df['Image Index'], df['Finding Labels']))
image_dir = 'Final/merged_images'
output_dir = 'Final/split_dataset'

os.makedirs(os.path.join(output_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test'), exist_ok=True)

all_images = [f for f in os.listdir(image_dir) if f.endswith('.png')]
print(f"Found {len(all_images)} images in directory")

matched_images = []
matched_labels = []
missing_count = 0

for img in all_images:
    key = img.strip().lower()
    if key in image_labels:
        matched_images.append(img)
        matched_labels.append(image_labels[key])
    else:
        missing_count += 1
print(f"Matched {len(matched_images)} images with labels")
print(f"{missing_count} images had no matching label")

if len(matched_images) == 0:
    raise ValueError("No images matched with labels in image_labels dictionary!")

X_train, X_temp, y_train, y_temp = train_test_split(matched_images, matched_labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

def process_dataset(images, labels, subset):
    subset_dir = os.path.join(output_dir, subset)
    label_file = os.path.join(output_dir, f'{subset}_labels.txt')
    
    for img in images:
        src = os.path.join(image_dir, img)
        dst = os.path.join(subset_dir, img)
        shutil.copyfile(src, dst)
    
    with open(label_file, 'w') as f:
        for img, label in zip(images, labels):
            f.write(f"{img} {label}\n")

process_dataset(X_train, y_train, 'train')
process_dataset(X_val, y_val, 'val')
process_dataset(X_test, y_test, 'test')

print("\nDataset splitting complete!")
print(f"Train: {len(X_train)} images")
print(f"Validation: {len(X_val)} images")
print(f"Test: {len(X_test)} images")
print(f"Total matched: {len(matched_images)} images")
print(f"Total unmatched: {missing_count} images")

Found 60990 images in directory
Matched 60990 images with labels
0 images had no matching label

Dataset splitting complete!
Train: 42693 images
Validation: 9148 images
Test: 9149 images
Total matched: 60990 images
Total unmatched: 0 images


In [4]:
import os
import pandas as pd
import numpy as np
from PIL import Image

CSV_PATH = "Final/dataset_columns.csv"
IMG_DIR = "Final/merged_images"

df = pd.read_csv(CSV_PATH)
df.columns = df.columns.str.strip()
print("Original columns:", df.columns.tolist())

total_rows = len(df)
unique_patients = df["Patient ID"].nunique()
duplicate_count = total_rows - unique_patients

print(f"Total rows: {total_rows}")
print(f"Unique Patient IDs: {unique_patients}")
print(f"Duplicate Patient ID entries: {duplicate_count}")

counts = df["Patient ID"].value_counts()
print("Patients with multiple images:")
print(counts[counts > 1])


Original columns: ['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID', 'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Unnamed: 11']
Total rows: 60990
Unique Patient IDs: 21597
Duplicate Patient ID entries: 39393
Patients with multiple images:
Patient ID
13670    86
10007    84
12834    81
1836     75
13993    74
         ..
17444     2
4467      2
19440     2
9652      2
13455     2
Name: count, Length: 9005, dtype: int64


In [7]:
import os
import pandas as pd
import numpy as np
from PIL import Image

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from sklearn.model_selection import GroupShuffleSplit

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

df = pd.read_csv("Final/dataset_columns.csv")
df.columns = df.columns.str.strip()
df["filename"] = df["Image Index"].astype(str)
df = df[df["filename"].isin(os.listdir("Final/merged_images"))].reset_index(drop=True)
df["label"] = df["View Position"].map({"PA":1, "AP":0})

gss1 = GroupShuffleSplit(n_splits = 1, test_size = 0.30, random_state = 42)
train_idx, temp_idx = next(gss1.split(df, groups=df["Patient ID"]))
train_df = df.iloc[train_idx]
temp_df = df.iloc[temp_idx]

gss2 = GroupShuffleSplit(n_splits = 1, test_size = 0.50, random_state = 42)
val_idx, test_idx = next(gss2.split(temp_df, groups=temp_df["Patient ID"]))
val_df = temp_df.iloc[val_idx]
test_df = temp_df.iloc[test_idx]

print(f"Splits → train:{len(train_df)}  val:{len(val_df)}  test:{len(test_df)}")

class XrayImageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.img_dir = img_dir
        self.filenames = df["filename"].tolist()
        self.labels = df["label"].tolist()
        self.transform = transform or T.Compose([T.ToTensor()])

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.img_dir, self.filenames[idx])).convert("RGB")
        x = self.transform(img)
        y = self.labels[idx]
        return x, y

batch_size = 32
train_loader = DataLoader(XrayImageDataset(train_df, "Final/merged_images"), batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(XrayImageDataset(val_df, "Final/merged_images"), batch_size, shuffle=False)
test_loader = DataLoader(XrayImageDataset(test_df, "Final/merged_images"), batch_size, shuffle=False)

class CNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.AdaptiveAvgPool2d(1)
        )
        self.classifier = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = correct = total = 0
    for X, y in loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)
    return total_loss/total, correct/total

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = correct = total = 0
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            total_loss += criterion(logits, y).item() * X.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return total_loss/total, correct/total

model = CNN(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 7
for epoch in range(1, epochs+1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch:02d} | train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} | val_acc: {val_acc:.4f}")

test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

save_path = "xray_cnn_model.pth"
torch.save(model.state_dict(), save_path)
print(f"Model weights saved to {save_path}")


Using device: mps
Splits → train:42695  val:9209  test:9086
Epoch 01 | train_loss: 0.3531 | train_acc: 0.8290 | val_loss: 0.1558 | val_acc: 0.9620
Epoch 02 | train_loss: 0.1126 | train_acc: 0.9695 | val_loss: 0.0813 | val_acc: 0.9785
Epoch 03 | train_loss: 0.0849 | train_acc: 0.9763 | val_loss: 0.0767 | val_acc: 0.9807
Epoch 04 | train_loss: 0.0758 | train_acc: 0.9789 | val_loss: 0.0567 | val_acc: 0.9878
Epoch 05 | train_loss: 0.0575 | train_acc: 0.9844 | val_loss: 0.0597 | val_acc: 0.9847
Epoch 06 | train_loss: 0.1090 | train_acc: 0.9669 | val_loss: 0.0584 | val_acc: 0.9835
Epoch 07 | train_loss: 0.0593 | train_acc: 0.9834 | val_loss: 0.0522 | val_acc: 0.9849
Test Loss: 0.0548 | Test Acc: 0.9862
Model weights saved to xray_cnn_model.pth
